In [1]:
%run Hooklog3.ipynb
import pandas as pd
import shutil
import os, pickle
import numpy as np
from tqdm import tqdm
from collections import Counter
Hooklog = Hooklog3

In [2]:
root_dir = "./data/Hooklog/trace_picked_fam/"
in_parseFirstPar = False # False=只拿API function name
fam_dirs = next(os.walk(root_dir))[1]
combine_dict = []
for fam in tqdm(fam_dirs):
    in_directory = root_dir + fam + '/'
    hl_list = next(os.walk(in_directory))[2] # get all filenames in the in_directory
    hl_list = [os.path.join(in_directory, f) for f in hl_list] # filepathname list
    hl_list = list(filter(lambda f: f.endswith(".hooklog"), hl_list)) # in case some non-hooklog file in the folder
    for file in hl_list:
        hl3 = Hooklog(file, in_parseFirstPar)
        hl_li = []
        for value in hl3.li:
            hl_li.append(value[1])
        combine_dict.append(hl_li)

100%|██████████| 14/14 [00:05<00:00,  2.67it/s]


In [3]:
# n_gram = 2
stride = 2
all_window_li = []
for hkl_li in combine_dict:
    for i in range(len(hkl_li)):
        if i%stride != 0:
            continue
        try:
            all_window_li.append((hkl_li[i],hkl_li[i+1])) #tri-gram:(hkl_li[i],hkl_li[i+1],hkl_li[i+2])
        except IndexError:
            break

In [4]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

def knee_point(length_li,k=5):
    """
    length_li: 長度list
    k: slope topK's knee points
    Return dict format (key is the answer)
    """
    length_dict = dict(Counter(length_li))
    sorted_dict = {k: v for k, v in sorted(length_dict.items(), key=lambda x: x[1])}
    all_items_num = sum(list(length_dict.values()))
#     for item in sorted_dict.items():
    all_keys = list(sorted_dict.keys())
    all_values = list(sorted_dict.values())
    slope_li = []
    for i in range(len(sorted_dict)):
        length1 = all_keys[i]
        try:
            length2 = all_keys[i+1]
        except IndexError:
            break
        value1 = sum(all_values[:i+1])
        value2 = sum(all_values[:i+2])
        slope = ((value2-value1)/all_items_num)/(length2-length1)
        slope_li.append(slope)
    change_rate_li = []
    for i in range(len(slope_li)):
        try:
            slope1 = slope_li[i]
            slope2 = slope_li[i+1]
        except IndexError:
            break
        change_rate_li.append(abs(slope2-slope1)) #陡變緩或是緩變陡的都一起算
    idx_li = sorted(range(len(change_rate_li)), key=lambda i: change_rate_li[i], reverse=True)[:k]
    return_dict = {}
    for idx in idx_li:
        return_dict[all_keys[idx+1]] = change_rate_li[idx]
    return pd.DataFrame.from_dict(return_dict,orient='index',columns=['knee_length'])
#     return return_dict        
        

In [5]:
statistic_gram = dict(Counter(all_window_li))
print(len(statistic_gram))
tri_dictionary_li = sorted(statistic_gram, key=statistic_gram.get,reverse=True)
tri_dictionary_li

332


[('RegOpenKey', 'RegOpenKey'),
 ('DeleteFile', 'DeleteFile'),
 ('CopyFile', 'CopyFile'),
 ('CloseHandle', 'CloseHandle'),
 ('RegOpenKey', 'RegQueryValue'),
 ('RegQueryValue', 'RegCloseKey'),
 ('RegQueryValue', 'RegQueryValue'),
 ('RegCloseKey', 'RegOpenKey'),
 ('LoadLibrary', 'LoadLibrary'),
 ('RegCloseKey', 'RegCloseKey'),
 ('CreateFile', 'CloseHandle'),
 ('CloseHandle', 'CreateFile'),
 ('RegEnumValue', 'RegEnumValue'),
 ('CreateFile', 'CopyFile'),
 ('RegOpenKey', 'RegCloseKey'),
 ('RegCloseKey', 'LoadLibrary'),
 ('RegQueryValue', 'RegOpenKey'),
 ('LoadLibrary', 'RegOpenKey'),
 ('CloseHandle', 'RegOpenKey'),
 ('RegCloseKey', 'RegCreateKey'),
 ('RegCloseKey', 'CloseHandle'),
 ('RegOpenKey', 'LoadLibrary'),
 ('LoadLibrary', 'CreateFile'),
 ('CloseHandle', 'LoadLibrary'),
 ('RegSetValue', 'RegCloseKey'),
 ('RegCloseKey', 'RegQueryValue'),
 ('CopyFile', 'CloseHandle'),
 ('RegCreateKey', 'RegQueryValue'),
 ('RegCreateKey', 'RegOpenKey'),
 ('RegOpenKey', 'RegEnumValue'),
 ('RegCreateKey', '

In [6]:
length_dist = list(statistic_gram.values())
stat_df = pd.DataFrame(sorted(length_dist,reverse=True),columns=['bi_num'],index=tri_dictionary_li)
stat_df.to_excel('./results/Hooklog/bigram_numbers.xlsx')
basic_statistics(length_dist)

,length
mean,2426.891566
std,10876.137430
mode,1.000000
min,1.000000
q1,2.000000
median,18.500000
q3,235.500000
max,92887.000000
iqr,233.500000
outlier,585.750000


In [7]:
tri_dict_enc = {}
for i,gram in enumerate(tri_dictionary_li):
    tri_dict_enc[gram] = i+1
pickle.dump(file=open(root_dir + "bi_dict_enc.pkl",'wb'),obj=tri_dict_enc)
tri_dict_enc

{('RegOpenKey', 'RegOpenKey'): 1,
 ('DeleteFile', 'DeleteFile'): 2,
 ('CopyFile', 'CopyFile'): 3,
 ('CloseHandle', 'CloseHandle'): 4,
 ('RegOpenKey', 'RegQueryValue'): 5,
 ('RegQueryValue', 'RegCloseKey'): 6,
 ('RegQueryValue', 'RegQueryValue'): 7,
 ('RegCloseKey', 'RegOpenKey'): 8,
 ('LoadLibrary', 'LoadLibrary'): 9,
 ('RegCloseKey', 'RegCloseKey'): 10,
 ('CreateFile', 'CloseHandle'): 11,
 ('CloseHandle', 'CreateFile'): 12,
 ('RegEnumValue', 'RegEnumValue'): 13,
 ('CreateFile', 'CopyFile'): 14,
 ('RegOpenKey', 'RegCloseKey'): 15,
 ('RegCloseKey', 'LoadLibrary'): 16,
 ('RegQueryValue', 'RegOpenKey'): 17,
 ('LoadLibrary', 'RegOpenKey'): 18,
 ('CloseHandle', 'RegOpenKey'): 19,
 ('RegCloseKey', 'RegCreateKey'): 20,
 ('RegCloseKey', 'CloseHandle'): 21,
 ('RegOpenKey', 'LoadLibrary'): 22,
 ('LoadLibrary', 'CreateFile'): 23,
 ('CloseHandle', 'LoadLibrary'): 24,
 ('RegSetValue', 'RegCloseKey'): 25,
 ('RegCloseKey', 'RegQueryValue'): 26,
 ('CopyFile', 'CloseHandle'): 27,
 ('RegCreateKey', 'Reg

### frequency encoding

In [28]:
# statistic_gram
statistic_gram_sort = {k: v for k, v in sorted(statistic_gram.items(), key=lambda x: x[1])}
max_num = max(length_dist)
min_num = min(length_dist)
length = len(statistic_gram_sort)
statistic_norm_enc = {}
for num,(i,v) in enumerate(statistic_gram_sort.items()):
    norm = 1-((num+1)/length) + 1/length
#     norm = 1 - ((v-min_num)/(max_num-min_num)*(1.0-1e-5)+1e-5)+1e-5 #1-(...)+1e-5:越常出現的越接近0
    statistic_norm_enc[i]=norm
statistic_norm_enc

{('RegCloseKey', 'WinHttpReadData'): 1.0,
 ('RegCreateKey', 'CreateProcess'): 0.996987951807229,
 ('WinHttpOpenRequest', 'RegCloseKey'): 0.9939759036144579,
 ('RegCreateKey', 'ExitProcess'): 0.9909638554216867,
 ('OpenProcess', 'WinExec'): 0.9879518072289157,
 ('RegCreateKey', 'RegOpenCurrentUser'): 0.9849397590361446,
 ('CopyFile', 'CreateRemoteThread'): 0.9819277108433735,
 ('DeleteFile', 'OpenProcess'): 0.9789156626506025,
 ('RegCloseKey', 'CreateThread'): 0.9759036144578314,
 ('CreateFile', 'CreateThread'): 0.9728915662650602,
 ('LoadLibrary', 'WinHttpOpenRequest'): 0.9698795180722892,
 ('CreateProcess', 'ExitProcess'): 0.9668674698795181,
 ('WinHttpConnect', 'CloseHandle'): 0.963855421686747,
 ('CloseHandle', 'WinHttpOpenRequest'): 0.960843373493976,
 ('WinHttpSendRequest', 'CreateFile'): 0.9578313253012049,
 ('RegDeleteValue', 'CreateFile'): 0.9548192771084337,
 ('CreateThread', 'WinExec'): 0.9518072289156627,
 ('RegDeleteKey', 'WinExec'): 0.9487951807228916,
 ('TerminateProcess'

In [29]:
root_dir = "./data/Hooklog/trace_picked_fam/"
in_parseFirstPar = False # False=只拿API function name
fam_dirs = next(os.walk(root_dir))[1]
all_enc_length = []
fam_names = []
for fam in tqdm(fam_dirs):
    in_directory = root_dir + fam + '/'
    hl_list = next(os.walk(in_directory))[2] # get all filenames in the in_directory
    hl_list = [os.path.join(in_directory, f) for f in hl_list] # filepathname list
    hl_list = list(filter(lambda f: f.endswith(".hooklog"), hl_list)) # in case some non-hooklog file in the folder
    for file in hl_list:
        hl3 = Hooklog(file, in_parseFirstPar)
        hl_li = []
        for i,value in enumerate(hl3.li):
            if i%stride!=0:
                continue
            try:
                hl_li.append((hl3.li[i][1],hl3.li[i+1][1])) #tri-gram: (hl3.li[i][1],hl3.li[i+1][1],hl3.li[i+2][1])
            except IndexError:
                break
        enc_li = []
        for n_gram in hl_li:
            try:
                enc_li.append(statistic_norm_enc[n_gram])
            except KeyError:
                print("Not in n-gram set:",n_gram)
                enc_li.append(0.0)
        all_enc_length.append(len(enc_li))
        fam_names.append(fam)
        np.save(file.replace('.trace.hooklog','.norm.npy'),np.array(enc_li))
stat_df = pd.DataFrame(all_enc_length,columns=['encoded_length'],index=fam_names)
stat_df.to_excel('./results/Hooklog/afterBigram_encLength.xlsx')
basic_statistics(all_enc_length)

100%|██████████| 14/14 [00:05<00:00,  2.57it/s]


,length
mean,257.915493
std,1335.793127
mode,175.000000
min,0.000000
q1,36.000000
median,105.000000
q3,183.000000
max,51669.000000
iqr,147.000000
outlier,403.500000


### convert hkl to tri-gram
* save as numpy to same path

In [8]:
root_dir = "./data/Hooklog/trace_picked_fam/"
in_parseFirstPar = False # False=只拿API function name
fam_dirs = next(os.walk(root_dir))[1]
all_enc_length = []
fam_names = []
for fam in tqdm(fam_dirs):
    in_directory = root_dir + fam + '/'
    hl_list = next(os.walk(in_directory))[2] # get all filenames in the in_directory
    hl_list = [os.path.join(in_directory, f) for f in hl_list] # filepathname list
    hl_list = list(filter(lambda f: f.endswith(".hooklog"), hl_list)) # in case some non-hooklog file in the folder
    for file in hl_list:
        hl3 = Hooklog(file, in_parseFirstPar)
        hl_li = []
        for i,value in enumerate(hl3.li):
            if i%stride!=0:
                continue
            try:
                hl_li.append((hl3.li[i][1],hl3.li[i+1][1])) #tri-gram: (hl3.li[i][1],hl3.li[i+1][1],hl3.li[i+2][1])
            except IndexError:
                break
        enc_li = []
        for n_gram in hl_li:
            try:
                enc_li.append(tri_dict_enc[n_gram])
            except KeyError:
                print("Not in n-gram set:",n_gram)
                enc_li.append(len(tri_dictionary_li)+1)
        all_enc_length.append(len(enc_li))
        fam_names.append(fam)
        np.save(file.replace('.trace.hooklog','.enc.npy'),np.array(enc_li))
#         combine_dict.append(hl_li)
stat_df = pd.DataFrame(all_enc_length,columns=['encoded_length'],index=fam_names)
stat_df.to_excel('./results/Hooklog/afterBigram_encLength.xlsx')
basic_statistics(all_enc_length)

100%|██████████| 14/14 [00:05<00:00,  2.61it/s]


,length
mean,257.915493
std,1335.793127
mode,175.000000
min,0.000000
q1,36.000000
median,105.000000
q3,183.000000
max,51669.000000
iqr,147.000000
outlier,403.500000


In [12]:
stat_df2 = knee_point(all_enc_length,k=50)
stat_df2[(stat_df2.index>1)&(stat_df2.index<180)]

,knee_length
106,0.004168
103,0.003922
33,0.003783
14,0.003404
31,0.003256
17,0.002929
18,0.001929
75,0.001867
20,0.001731
23,0.001715
